<a href="https://colab.research.google.com/github/ronald-hk-chung/ztm_pytorch/blob/master/05_pytorch_going_modular_exercise_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05. PyTorch Going Modular Exercises

Welcome to the 05. PyTorch Going Modular exercise template notebook.

There are several questions in this notebook and it's your goal to answer them by writing Python and PyTorch code.

> **Note:** There may be more than one solution to each of the exercises, don't worry too much about the *exact* right answer. Try to write some code that works first and then improve it if you can.

## Resources and solutions

* These exercises/solutions are based on [section 05. PyTorch Going Modular](https://www.learnpytorch.io/05_pytorch_going_modular/) of the Learn PyTorch for Deep Learning course by Zero to Mastery.

**Solutions:**

Try to complete the code below *before* looking at these.

* See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/ijgFhMK3pp4).
* See an example [solutions notebook for these exercises on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/solutions/05_pytorch_going_modular_exercise_solutions.ipynb).

## 1. Turn the code to get the data (from section 1. Get Data) into a Python script, such as `get_data.py`.

* When you run the script using `python get_data.py` it should check if the data already exists and skip downloading if it does.
* If the data download is successful, you should be able to access the `pizza_steak_sushi` images from the `data` directory.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pathlib
module_path = pathlib.Path('modules')
module_path.mkdir(parents=True, exist_ok=True)

In [3]:
%%writefile modules/walk_through_dir.py
'''
Contains code to walk through directories
'''
import os

def walk_through_dir(dir_path):
  """
  Walks through dir_path returning its contents.
  Args:
    dir_path (str): target directory

  Returns:
    A print out of:
      number of subdiretories in dir_path
      number of images (files) in each subdirectory
      name of each subdirectory
  """
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

Writing modules/walk_through_dir.py


In [4]:
%%writefile modules/unzip_image.py
'''
Functions to Unzip --zippath to --target
'''
import os
import pathlib
import shutil
import zipfile
import argparse
import walk_through_dir

# Setting up the arg parser
parser = argparse.ArgumentParser()
parser.add_argument('--target', type=str, required=True)
parser.add_argument('--zippath', type=str, required=True)
args = parser.parse_args()

# Creating target directory
data_path = pathlib.Path(args.target)
data_path.mkdir(parents=True, exist_ok=True)
print(f'{data_path} created')

# Copying and zipref to target
shutil.copy2(src=str(args.zippath),
             dst=str(data_path / 'zipref.zip'))
print(f'Copied... {args.zippath} --> {data_path}')

with zipfile.ZipFile(data_path / 'zipref.zip') as zip_ref:
  zip_ref.extractall(data_path)
  print(f'Unzipped Files to {data_path}')

# Removing zipref
os.remove(data_path / 'zipref.zip')

# Walkthrough data_path
walk_through_dir.walk_through_dir(data_path)

Writing modules/unzip_image.py


In [5]:
!python modules/unzip_image.py --target 'data/pizza_steak_sushi' --zippath '/content/drive/MyDrive/ML/ZTM PyTorch/data/pizza_steak_sushi_100_percent.zip'

data/pizza_steak_sushi created
Copied... /content/drive/MyDrive/ML/ZTM PyTorch/data/pizza_steak_sushi_100_percent.zip --> data/pizza_steak_sushi
Unzipped Files to data/pizza_steak_sushi
There are 2 directories and 0 images in 'data/pizza_steak_sushi'.
There are 3 directories and 0 images in 'data/pizza_steak_sushi/test'.
There are 0 directories and 194 images in 'data/pizza_steak_sushi/test/pizza'.
There are 0 directories and 209 images in 'data/pizza_steak_sushi/test/sushi'.
There are 0 directories and 197 images in 'data/pizza_steak_sushi/test/steak'.
There are 3 directories and 0 images in 'data/pizza_steak_sushi/train'.
There are 0 directories and 806 images in 'data/pizza_steak_sushi/train/pizza'.
There are 0 directories and 791 images in 'data/pizza_steak_sushi/train/sushi'.
There are 0 directories and 803 images in 'data/pizza_steak_sushi/train/steak'.


## 2. Use [Python's `argparse` module](https://docs.python.org/3/library/argparse.html) to be able to send the `train.py` custom hyperparameter values for training procedures.
* Add an argument flag for using a different:
  * Training/testing directory
  * Learning rate
  * Batch size
  * Number of epochs to train for
  * Number of hidden units in the TinyVGG model
    * Keep the default values for each of the above arguments as what they already are (as in notebook 05).
* For example, you should be able to run something similar to the following line to train a TinyVGG model with a learning rate of 0.003 and a batch size of 64 for 20 epochs: `python train.py --learning_rate 0.003 batch_size 64 num_epochs 20`.
* **Note:** Since `train.py` leverages the other scripts we created in section 05, such as, `model_builder.py`, `utils.py` and `engine.py`, you'll have to make sure they're available to use too. You can find these in the [`going_modular` folder on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/going_modular/going_modular).

In [6]:
%%writefile modules/data_setup.py
'''
Contains functionality for creating Pytorch DataLoaders for
image classification data following torchvision.datasets.ImageFolder
'''

import os
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

NUM_WORKERS = os.cpu_count()

def create_dataloaders(train_dir: str,
                       test_dir: str,
                       transform: transforms.Compose,
                       batch_size: int,
                       num_workers: int=NUM_WORKERS) -> tuple[DataLoader, DataLoader, list[str], dict[str, int]]:
  """Creates training and testing DataLoaders.

  Takes in a training directory and testing directory path and turns
  them into PyTorch Datasets and then into PyTorch DataLoaders.

  Args:
    train_dir: Path to training directory.
    test_dir: Path to testing directory.
    transform: torchvision transforms to perform on training and testing data.
    batch_size: Number of samples per batch in each of the DataLoaders.
    num_workers: An integer for number of workers per DataLoader.

  Returns:
    A tuple of (train_dataloader, test_dataloader, class_names).
    Where class_names is a list of the target classes.
    Example usage:
      train_dataloader, test_dataloader, class_names = \
        = create_dataloaders(train_dir=path/to/train_dir,
                             test_dir=path/to/test_dir,
                             transform=some_transform,
                             batch_size=32,
                             num_workers=4)
  """
  # Use ImageFolder to create datasets
  train_data = datasets.ImageFolder(root=train_dir, transform=transform)
  test_data = datasets.ImageFolder(root=test_dir, transform=transform)

  # Get class_names and class_dict
  class_names = train_data.classes
  class_dict = train_data.class_to_idx

  # Turn images into DataLoaders
  train_dataloader = DataLoader(dataset=train_data,
                                batch_size=batch_size,
                                shuffle=True,
                                num_workers=num_workers,
                                pin_memory=True)
  test_dataloader = DataLoader(dataset=test_data,
                               batch_size=batch_size,
                               shuffle=False,
                               num_workers=num_workers,
                               pin_memory=True)

  return train_dataloader, test_dataloader, class_names, class_dict

Writing modules/data_setup.py


In [7]:
%%writefile modules/model_builder.py
"""
Contains PyTorch model code to instantiate a TinyVGG model.
"""
import torch
from torch import nn

class TinyVGG(nn.Module):
  """Creates the TinyVGG architecture.

  Replicates the TinyVGG architecture from the CNN explainer website in PyTorch.
  See the original architecture here: https://poloclub.github.io/cnn-explainer/

  Args:
  input_shape: An integer indicating number of input channels.
  hidden_units: An integer indicating number of hidden units between layers.
  output_shape: An integer indicating number of output units.
  """
  def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
      super().__init__()
      self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                      stride=2)
      )
      self.conv_block_2 = nn.Sequential(
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
        nn.ReLU(),
        nn.Conv2d(hidden_units, hidden_units, kernel_size=3, padding=0),
        nn.ReLU(),
        nn.MaxPool2d(2)
      )
      self.classifier = nn.Sequential(
        nn.Flatten(),
        # Where did this in_features shape come from?
        # It's because each layer of our network compresses and changes the shape of our inputs data.
        nn.Linear(in_features=hidden_units*13*13,
                  out_features=output_shape)
      )

  def forward(self, x: torch.Tensor):
      x = self.conv_block_1(x)
      x = self.conv_block_2(x)
      x = self.classifier(x)
      return x

Writing modules/model_builder.py


In [8]:
try:
  import torchmetrics
except:
  !pip install torchmetrics
  import torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 16.3 MB/s eta 0:00:00


In [9]:
%%writefile modules/engine.py
"""
Contains functions for training and testing a PyTorch model.
"""

import torch
import torchmetrics
from tqdm.auto import tqdm


def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device: torch.device) -> tuple[float, float]:
  '''Trains a PyTorch model for a single epoch.

  Turns a target PyTorch model to training mode and then
  runs through all of the required training steps (forward
  pass, loss calculation, optimizer step).

  Args:
  model: A PyTorch model to be trained.
  dataloader: A DataLoader instance for the model to be trained on.
  loss_fn: A PyTorch loss function to minimize.
  optimizer: A PyTorch optimizer to help minimize the loss function.
  device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
  A tuple of training loss and training accuracy metrics.
  In the form (train_loss, train_accuracy). For example:

  (0.1112, 0.8743)
  '''
  # Put model in train mode
  model.train()

  # Setup train loss and train accuracy value
  train_loss, train_acc = 0, 0

  # Loop through data loader data batches
  for batch, (X, y) in enumerate(dataloader):
    # Send data to target device
    X, y = X.to(device), y.to(device)

    # Forward Pass
    y_logits = model(X)

    # Calculate and accumulate train_loss/train_acc
    loss = loss_fn(y_logits, y)
    train_loss += loss.item()
    acc = (y_logits.softmax(dim=1).argmax(dim=1)==y).sum().item()/len(y)
    train_acc += acc

    # Optimizer zero grad
    optimizer.zero_grad()

    # Loss backward
    loss.backward()

    # Optimzer step
    optimizer.step()

  # Caclulate and accumulate accuracy metrics across all batches
  train_loss /= len(dataloader)
  train_acc /= len(dataloader)

  return train_loss, train_acc


def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device: torch.device) -> tuple[float, float]:
  """Tests a PyTorch model for a single epoch.

  Turns a target PyTorch model to "eval" mode and then performs
  a forward pass on a testing dataset.

  Args:
  model: A PyTorch model to be tested.
  dataloader: A DataLoader instance for the model to be tested on.
  loss_fn: A PyTorch loss function to calculate loss on the test data.
  device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
  A tuple of testing loss and testing accuracy metrics.
  In the form (test_loss, test_accuracy). For example:

  (0.0223, 0.8985)
  """
  # Put model to eval mode
  model.eval()

  # Setup test loss and test accuracy values
  test_loss, test_acc = 0, 0

  # Turn on inference context manager
  with torch.inference_mode():
    # Loop through DataLoader batches
    for batch, (X, y) in enumerate(dataloader):
      # Send data to target device
      X, y = X.to(device), y.to(device)

      # Forward pass
      y_logits = model(X)

      # Calculate and accumulate loss/acc
      loss = loss_fn(y_logits, y)
      test_loss += loss.item()
      acc = (y_logits.softmax(dim=1).argmax(dim=1)==y).sum().item()/len(y)
      test_acc += acc

  # Adjust metrics to get average loss and accuracy per batch
  test_loss /= len(dataloader)
  test_acc /= len(dataloader)

  return test_loss, test_acc


def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int,
          device: torch.device) -> dict[str, list[float]]:
  """Trains and tests a PyTorch model.

  Passes a target PyTorch models through train_step() and test_step()
  functions for a number of epochs, training and testing the model
  in the same epoch loop.

  Calculates, prints and stores evaluation metrics throughout.

  Args:
  model: A PyTorch model to be trained and tested.
  train_dataloader: A DataLoader instance for the model to be trained on.
  test_dataloader: A DataLoader instance for the model to be tested on.
  optimizer: A PyTorch optimizer to help minimize the loss function.
  loss_fn: A PyTorch loss function to calculate loss on both datasets.
  epochs: An integer indicating how many epochs to train for.
  device: A target device to compute on (e.g. "cuda" or "cpu").

  Returns:
  A dictionary of training and testing loss as well as training and
  testing accuracy metrics. Each metric has a value in a list for
  each epoch.
  In the form: {train_loss: [...],
            train_acc: [...],
            test_loss: [...],
            test_acc: [...]}
  For example if training for epochs=2:
            {train_loss: [2.0616, 1.0537],
            train_acc: [0.3945, 0.3945],
            test_loss: [1.2641, 1.5706],
            test_acc: [0.3400, 0.2973]}
  """
  # Create empty results dictionary
  results = {'train_loss': [],
             'train_acc': [],
             'test_loss': [],
             'test_acc': []}

  #Loop through training and testing steps for a number of epochs
  for epoch in tqdm(range(epochs), total=epochs):
    train_loss, train_acc = train_step(model=model,
                                       dataloader=train_dataloader,
                                       loss_fn=loss_fn,
                                       optimizer=optimizer,
                                       device=device)
    test_loss, test_acc = test_step(model=model,
                                     dataloader=test_dataloader,
                                     loss_fn=loss_fn,
                                     device=device)

    # Print out results
    print(
        f'\nEpoch: {epoch+1} |'
        f'train_loss: {train_loss:.4f} |'
        f'train_acc: {train_acc:.4f} |'
        f'test_loss: {test_loss:.4f} |'
        f'test_acc: {test_acc:.4f}'
    )

    # Update results dictionary
    results['train_loss'].append(train_loss)
    results['train_acc'].append(train_acc)
    results['test_loss'].append(test_loss)
    results['test_acc'].append(test_acc)

  #Return the filled results
  return results

Writing modules/engine.py


In [10]:
%%writefile modules/utils.py
"""
Contains various utility functions for PyTorch model training and saving.
"""
import torch
from pathlib import Path
def save_model(model: torch.nn.Module,
              target_dir: str,
              model_name: str):
  """Saves a PyTorch model to a target directory.

  Args:
  model: A target PyTorch model to save.
  target_dir: A directory for saving the model to.
  model_name: A filename for the saved model. Should include
    either ".pth" or ".pt" as the file extension.

  Example usage:
  save_model(model=model_0,
              target_dir="models",
              model_name="05_going_modular_tingvgg_model.pth")
  """
  # Create target directory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True, exist_ok=True)

  # Create model save path
  model_save_path = target_dir_path / model_name

  # Save the model state_dict()
  print(f'Saving model to: {model_save_path}')
  torch.save(obj=model.state_dict(),
             f=model_save_path)

Writing modules/utils.py


In [11]:
try:
  import torchinfo
except:
  !pip install torchinfo
  import torchinfo

In [57]:
%%writefile modules/train.py
'''
Trains a Pytorch image classification model using device-agnostic code.
'''
import os
import torch
from torchvision import transforms
import data_setup, engine, model_builder, utils
import torchinfo
import argparse

# Setting up the arg parser
parser = argparse.ArgumentParser()
parser.add_argument('--epochs', type=int, required=True)
parser.add_argument('--batch_size', type=int, required=True)
parser.add_argument('--hidden_units', type=int, required=True)
parser.add_argument('--learning_rate', type=float, required=True)
parser.add_argument('--data_dir', type=str, required=True)
parser.add_argument('--transform_resize', type=int, required=True)
parser.add_argument('--savemodel_name', type=str, required=True)
args = parser.parse_args()


# Setup hyperparameters
NUM_EPOCHS = args.epochs
BATCH_SIZE = args.batch_size
HIDDEN_UNITS = args.hidden_units
LEARNING_RATE = args.learning_rate
TRANSFORM_RESIZE = args.transform_resize

# Setup directories
data_dir = args.data_dir
train_dir = data_dir + '/train'
test_dir = data_dir + '/test'
savemodel_name = args.savemodel_name

# Setup target device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Create transforms
data_transform = transforms.Compose([transforms.Resize(size=(TRANSFORM_RESIZE, TRANSFORM_RESIZE)),
                                     transforms.ToTensor()])

# Create DataLoaders with help from data_setup.py
train_dataloader, test_dataloader, class_names, class_dict = \
data_setup.create_dataloaders(train_dir=train_dir,
                              test_dir=test_dir,
                              transform=data_transform,
                              batch_size=BATCH_SIZE)

# Create model with help from model_builder.py
model = model_builder.TinyVGG(input_shape=3,
                              hidden_units=HIDDEN_UNITS,
                              output_shape=len(class_names)
                              ).to(device)

torchinfo.summary(model, input_size=next(iter(train_dataloader))[0].shape)

# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(),
                             lr=LEARNING_RATE)

# Start training with help from engine.py
engine.train(model=model,
             train_dataloader=train_dataloader,
             test_dataloader=test_dataloader,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)

# Save the model with help from utils.py
utils.save_model(model=model,
                 target_dir='models',
                 model_name=savemodel_name)


Overwriting modules/train.py


In [116]:
!python modules/train.py --data_dir 'data/pizza_steak_sushi' --savemodel_name 'pizza_steak_sushi_tinyvgg.pth' \
--epochs 20 --batch_size 32 --hidden_units 20 --learning_rate=0.001 --transform_resize 64

Layer (type:depth-idx)                   Output Shape              Param #
TinyVGG                                  [32, 3]                   --
├─Sequential: 1-1                        [32, 20, 30, 30]          --
│    └─Conv2d: 2-1                       [32, 20, 62, 62]          560
│    └─ReLU: 2-2                         [32, 20, 62, 62]          --
│    └─Conv2d: 2-3                       [32, 20, 60, 60]          3,620
│    └─ReLU: 2-4                         [32, 20, 60, 60]          --
│    └─MaxPool2d: 2-5                    [32, 20, 30, 30]          --
├─Sequential: 1-2                        [32, 20, 13, 13]          --
│    └─Conv2d: 2-6                       [32, 20, 28, 28]          3,620
│    └─ReLU: 2-7                         [32, 20, 28, 28]          --
│    └─Conv2d: 2-8                       [32, 20, 26, 26]          3,620
│    └─ReLU: 2-9                         [32, 20, 26, 26]          --
│    └─MaxPool2d: 2-10                   [32, 20, 13, 13]          --
├─Seq

## 3. Create a Python script to predict (such as `predict.py`) on a target image given a file path with a saved model.

* For example, you should be able to run the command `python predict.py some_image.jpeg` and have a trained PyTorch model predict on the image and return its prediction.
* To see example prediction code, check out the [predicting on a custom image section in notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/#113-putting-custom-image-prediction-together-building-a-function).
* You may also have to write code to load in a trained model.

In [117]:
%%writefile modules/predict.py
"""
Function to predict given filepath and modelpath base on TinyVGG
"""
import pathlib
import model_builder
import torch
import torchvision
from torchvision import transforms
import matplotlib.pyplot as plt
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--file', type=str, required=True)
parser.add_argument('--model', type=str, required=True)
args = parser.parse_args()

FILE_PATH = args.file
MODEL_PATH = args.model

# Create model with help from model_builder.py
model = model_builder.TinyVGG(input_shape=3,
                              hidden_units=10,
                              output_shape=3
                              )

# Load in the saved state_dict()
model.load_state_dict(torch.load(f=MODEL_PATH))

# Transform image using transform
data_transform = transforms.Compose([transforms.Resize(size=(64, 64), antialias=True)])
image_tensor = torchvision.io.read_image(str(FILE_PATH))
image_transformed = data_transform(image_tensor.type(torch.float32))

# Make prediction
model.eval()
with torch.inference_mode():
  logits = model(image_transformed.unsqueeze(dim=0))
print(logits)
print(logits.softmax(dim=1))
pred = logits.softmax(dim=1).argmax(dim=1).item()
class_names = ['pizza', 'steak', 'sushi']
pred_class = class_names[pred]
pred_prob = logits.softmax(dim=1).squeeze()[pred]
print(f'Prediction is {pred_class} with probabilities of {pred_prob*100}%')

Overwriting modules/predict.py


In [120]:
!python modules/predict.py \
--model 'models/pizza_sushi_steak_tinyvgg_model.pth' \
--file 'data/pizza_steak_sushi/test/sushi/106904.jpg'

tensor([[-325.2700,  -93.5870,  186.8926]])
tensor([[0., 0., 1.]])
Prediction is sushi with probabilities of 100.0%


In [125]:
import shutil
import pathlib
shutil.copytree(src=pathlib.Path('/content/modules'),
                dst=pathlib.Path('/content/drive/MyDrive/ML/ZTM PyTorch/modules'))

PosixPath('/content/drive/MyDrive/ML/ZTM PyTorch/modules')